In [11]:
import csv
import json
import numpy as np
import pandas as pd
import re
import utils
from collections import Counter
import swifter


In [18]:
df_orig = pd.read_csv("xandr_segments.csv", sep=",") # load data
# rename columns
df_orig = df_orig.rename(columns={"Segment Name": "name", "Data Provider Name": "provider_name","Data Provider ID": "provider_id","Segment ID" : "id"})
df_orig["name_list"] = np.nan # introduce empty column for itemized segment name
df_orig["data_source"] = np.nan # data source in brackets at the end of a segment name

In [19]:
provider_ids = df_orig["provider_id"].unique()
provider_names = df_orig["provider_name"].unique()

In [20]:
Counter(df_orig["provider_name"]).most_common(50)

[('Audiences by Oracle (BlueKai, Datalogix, AddThis)', 132645),
 ('LiveRamp Data Store', 82363),
 ('Grapeshot', 73569),
 ('Nielsen Marketing Cloud', 65610),
 ('Eyeota', 53526),
 ('Factual Inc', 29208),
 ('Oracle Customs (1st, BlueKai, Datalogix, AddThis)', 26288),
 ('Adsquare (Data Provider)', 15246),
 ('Dstillery', 12630),
 ('Skydeo, Inc.', 11972),
 ('Semcasting (Data Provider)', 10988),
 ('OwnerIQ Inc.', 10027),
 ('Lotame', 9523),
 ('Peer39', 7961),
 ('Retargetly', 7233),
 ('AlikeAudience, Inc.', 7075),
 ('KBM Group', 6765),
 ('comScore Inc.', 6451),
 ('ZeoTap', 5367),
 ('AdAdvisor by Neustar', 4992),
 ('Bombora Inc.', 4882),
 ('Tru Optik (CTV)', 4127),
 ('Epsilon Data Management, LLC.', 3812),
 ('HYP PTY LTD', 3661),
 ('ShareThis Inc.', 3454),
 ('VisualDNA', 2988),
 ('Tail Target (Data Provider)', 2966),
 ('Acxiom', 2886),
 ('Adstra, LLC', 2882),
 ('Xandr', 2706),
 ('Accordant Media LLC', 2683),
 ('The ADEX GmbH', 2628),
 ('Navegg', 2492),
 ('ALLIANCE GRAVITY DATA MEDIA', 2437),
 ('

In [21]:
utils.df_apply_function(df_orig, "name", utils.clean_segment_name) # clean all segment names

In [22]:
# itemize all segment names

data_source_re = re.compile(r"\([\w ]{3,25}\)$")

df_failed_segmentation = pd.DataFrame(columns=df_orig.columns)

def extract_data_source(row):
    name = row["name"]
    if data_source_match := re.search(data_source_re, name): # search for a data source at the end of the segment name
            row["data_source"] = data_source_match.group(0)[1:-1] # enter the data source into the corrseponding column (without brackets)
            row["name"] = name[:data_source_match.start(0)] # crop the existing segment name
    
    return row

df_orig = df_orig.swifter.apply(extract_data_source, axis=1)


pruning_re = re.compile(r"\(en\).+\(es\).+\(pt\)")
def prune_segname(row):
    name = row["name"]
    if match := re.search(pruning_re, name):
         row["name"] = name[:match.start(0)]
    return row


df_orig = df_orig.swifter.apply(prune_segname, axis=1)


Pandas Apply:   0%|          | 0/651463 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/651463 [00:00<?, ?it/s]

In [23]:
df_orig["name_list"] = df_orig.swifter.apply(lambda row: utils.itemize_segment_name(row["name"]), axis = 1)

Pandas Apply:   0%|          | 0/651463 [00:00<?, ?it/s]

In [24]:
utils.useless_segments_re = re.compile(r"\[null\]|\Atest|[^a-z]+test\Z| test |automation\d{5,10}")


In [25]:
useless_segment_indices = df_orig.swifter.apply(lambda x: bool(re.search(utils.useless_segments_re, x["name"])), axis = 1)

df_useless_segments = df_orig[useless_segment_indices].reset_index(drop=True)
df_useless_segments.to_csv("useless_segments.csv")

df_filtered = df_orig[useless_segment_indices.__invert__()]
df_filtered.to_csv('xandr_segments_itemized.csv')
df_filtered.to_json('xandr_segments_itemized.json')

Pandas Apply:   0%|          | 0/651463 [00:00<?, ?it/s]

In [26]:
df_failed_segmentation = df_filtered[df_filtered.swifter.apply(lambda x: len(x["name_list"]) < 2, axis = 1)]
df_failed_segmentation = df_failed_segmentation.reset_index(drop=True)
df_failed_segmentation.to_csv('xandr_segments_failed_itemization.csv')

Pandas Apply:   0%|          | 0/649110 [00:00<?, ?it/s]